In [1]:
!pip install --quiet transformers evaluate datasets baal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.6/419.6 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install --quiet --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 5.0 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/RuSportSum/result_news.csv" /content/result_news.csv
!cp -r "/content/drive/MyDrive/RuSportSum/bald" "/content/bald"

In [5]:
import os
import random
from copy import deepcopy

import pandas as pd
import numpy as np
import torch.backends
import transformers
import evaluate

from datasets import load_dataset
from tqdm import tqdm
from transformers import BertForSequenceClassification
from transformers import BertTokenizer, TrainingArguments
from transformers import set_seed

from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    PretrainedConfig,
    set_seed,
    Trainer,
    TrainingArguments
)

transformers.utils.logging.set_verbosity_warning()

from baal.active import get_heuristic
from baal.active.dataset.nlp_datasets import (
    active_huggingface_dataset,
    HuggingFaceDatasets,
)
from baal.bayesian.dropout import patch_module
from baal.transformers_trainer_wrapper import BaalTransformersTrainer
from baal.active import get_heuristic, ActiveLearningDataset
from baal.active.dataset.base import Dataset

from typing import List
from pprint import pprint

SEED = 3

random.seed(SEED)
torch.manual_seed(SEED)

# Set tranformer seed to ensure that initial weights are identical
set_seed(SEED)

In [6]:
data = pd.read_csv("/content/result_news.csv")
data

,id,name,ctime,body,match_id
0,7006398,Хиддинк постетит матч «Астон Виллы» и ЦСКА,2009-02-18 03:17:23,Главный тренер сборной России и «Челси» Гус Хи...,18936
1,7010916,Кубок УЕФА. 1/16 финала. Первые матчи. «Зенит»...,2009-02-18 15:57:00,Кубок УЕФА 1/16 финала Первые матчи,18935
2,7010916,Кубок УЕФА. 1/16 финала. Первые матчи. «Зенит»...,2009-02-18 15:57:00,Кубок УЕФА 1/16 финала Первые матчи,18936
3,7012099,«Зенит» и «Штутгарт» могут сыграть красным мяч...,2009-02-18 17:33:12,Делегат УЕФА Зуне Хелльстремер сегодня утром о...,18935
4,7012532,В кассах «Петровского» осталось 1500 билетов н...,2009-02-18 18:03:35,"Как стало известно корреспонденту Sports.ru, н...",18935
...,...,...,...,...,...
24643,1040626745,НХЛ. «Питтсбург» – «Сан-Хосе». Ньето заменит З...,2016-06-01 20:52:37,Нападающий «Сан-Хосе» Мэттью Ньето войдет в со...,1104866
24644,1040636713,НХЛ. Форвард «Питтсбурга» Шери забил первый по...,2016-06-02 08:20:10,Во втором матче финала Кубка Стэнли «Питтсбург...,1104866
24645,1040636981,НХЛ. Мюррей одержал 13-ю победу в дебютном пле...,2016-06-02 08:41:16,Во втором матче финала Кубка Стэнли «Питтсбург...,1104866
24646,1040637171,НХЛ. Звено Хагелин – Бонино – Кессел суммарно ...,2016-06-02 08:52:19,Во втором матче финала Кубка Стэнли «Питтсбург...,1104866


In [7]:
data['name_body'] = data['name'] + ". " + data['body']

In [8]:
good_news = [1060,
 22621,
 11364,
 14128,
 19302,
 22051,
 18101,
 8947,
 6018,
 6550,
 6253,
 6373,
 10976,
 8931,
 17738,
 21070,
 18414,
 23319,
 12509,
 14520,
 3512,
 2456,
 22250,
 18979,
 19332,
 10616,
 17758,
 3411,
 2344,
 20748,
 15237,
 1111,
 4145,
 9149,
 21199,
 10295,
 2791,
 7517,
 13657,
 6719,
 13544,
 14203,
 16326,
 15394,
 17185,
 10817,
 17656,
 435,
 4015,
 13455,
 13381,
 22302,
 14718,
 8791,
 758,
 20008,
 16752,
 22804,
 11631,
 5326,
 15371,
 19897,
 8712,
 9565,
 2896,
 9608,
 13592,
 20209,
 23978,
 15157,
 15931,
 21096,
 20336,
 11906,
 6679,
 21626,
 22963,
 21357,
 20522,
 11043,
 12817,
 14044,
 20180,
 18930,
 12497,
 19354,
 21153,
 7484,
 21667,
 4635,
 15175,
 17107,
 14308,
 5723,
 1509,
 19982,
 24374,
 11121,
 20016,
 6384,
 20617,
 7885,
 8532,
 9646,
 17615,
 17845,
 18947,
 12605,
 241,
 13903,
 7088,
 17821,
 1043,
 1445,
 3498,
 24170,
 15200,
 1211,
 21831,
 23507,
 13535,
 23004,
 6841,
 23067,
 16849,
 12841,
 13347,
 15524,
 24099,
 5093,
 15225,
 24220,
 2035,
 9470,
 9528,
 13841,
 8752,
 13326,
 16380,
 5932,
 9580,
 8384,
 22563,
 13247,
 11874,
 17755,
 16834,
 6296,
 1323,
 14672,
 20668,
 12909,
 12483,
 357,
 16527,
 23406,
 8090,
 13705,
 19981,
 5855,
 14679,
 7942,
 8632,
 20171,
 12138,
 21383,
 20376,
 9950,
 19614,
 16968,
 9142,
 4546,
 21774,
 18000,
 8096,
 10893,
 22425,
 24182,
 9564,
 20652,
 9238,
 18019,
 14299,
 18119,
 13971,
 14598,
 8047,
 12668,
 17361,
 8258,
 21296,
 12647,
 14810,
 3054,
 2604,
 16274,
 7638,
 20686,
 9492,
 17846,
 22373,
 490,
 9686,
 19854,
 1434,
 6374,
 7342,
 20468,
 18940,
 11335,
 12162,
 6754,
 18945,
 6489,
 18350,
 18654,
 9791,
 13974,
 13236,
 14947,
 15527,
 13109,
 20852,
 18170,
 5851,
 3712,
 14314,
 11246,
 15681,
 13002,
 22194,
 21566,
 19852,
 1931,
 16248,
 7564,
 3842,
 12703,
 14965,
 15890,
 14362,
 13277,
 4001,
 8848,
 17385,
 22186,
 2017,
 13475,
 15811,
 3868,
 22634,
 6716,
 19727,
 10483,
 10651,
 24600,
 829,
 19020,
 23845,
 22470,
 10187,
 4276,
 7900,
 14757,
 15416,
 16893,
 20311,
 24000,
 16525,
 4955,
 9351,
 9505,
 20147,
 14012,
 23201,
 18389,
 15067,
 19912,
 12863,
 17621,
 17276,
 2499,
 20995,
 1120,
 7486,
 3660,
 16767,
 9175,
 20199,
 1192,
 8599,
 23444,
 17899,
 2696,
 21489,
 21420,
 12155,
 8629,
 16874,
 14334,
 21072,
 19114,
 7616,
 22367,
 1401,
 14077,
 24159,
 19983,
 23736,
 5514,
 6453,
 21338,
 5433,
 17104,
 14042,
 10759,
 8029,
 13482,
 10856,
 22761,
 10626,
 6259,
 23220,
 23486,
 15087,
 11613,
 9735,
 19478,
 15397,
 17134,
 9764,
 22275,
 16652,
 23163,
 10183,
 2903,
 14384,
 7126,
 12768,
 14381,
 8458,
 9881,
 11689,
 19832,
 6784,
 1978,
 12311,
 9222,
 18932,
 1147,
 10522,
 10321,
 16039,
 21460,
 20579,
 21996,
 17640,
 12807,
 11056,
 2673,
 11290,
 8841,
 10955,
 8483,
 22786,
 19483,
 21262,
 13426,
 22949,
 23870,
 1748,
 15696,
 16703,
 11849,
 2924,
 24511,
 17439,
 21777,
 1183,
 6962,
 7074,
 16462,
 15035,
 10028,
 23372,
 9745,
 3050,
 1242,
 19218,
 18075,
 478,
 12263,
 8832,
 16217,
 21933,
 9793,
 8484,
 13966,
 16684,
 17557,
 17279,
 13591,
 1287,
 3377,
 14745,
 2429,
 5936,
 4102,
 12797,
 7898,
 18555,
 7784,
 11478,
 20705,
 17485,
 22197,
 5202,
 1615,
 6566,
 15759,
 8721,
 10995,
 23322,
 18193,
 21865,
 5634,
 24187,
 5901,
 11147,
 23295,
 6957,
 22002,
 13418,
 17446,
 1943,
 5328,
 11492,
 18894,
 23926,
 8160,
 13723,
 19714,
 5507,
 12731,
 1400,
 6267,
 11504,
 12265,
 24079,
 17706,
 13639,
 11459,
 24237,
 4890,
 18548,
 19827,
 8394,
 8952,
 15773,
 14755,
 19059,
 10080,
 15410,
 11936,
 6084,
 7761,
 3463,
 15575,
 16186,
 4475,
 16422,
 13702,
 3796,
 12478,
 3065,
 17567,
 4939,
 11703,
 13647,
 24007,
 20541,
 18500,
 7169,
 18416,
 18814,
 21043,
 11253,
 11168,
 19553,
 5558,
 12023,
 15778,
 22863,
 12631,
 8167,
 14891,
 9728,
 19032,
 12137,
 18174,
 12627,
 3363,
 16719,
 2709,
 20939,
 21081,
 12782,
 17416,
 20461,
 22576,
 19439,
 1511,
 21454,
 4721,
 12040,
 23118,
 20453,
 16311,
 16935,
 4045,
 20334,
 18036,
 23396,
 21793,
 11162,
 4051,
 10521,
 2874,
 12364,
 8287,
 14297,
 2037,
 5320,
 16732,
 9451,
 1325,
 22056,
 8592,
 7992,
 3232,
 9295,
 16478,
 5177,
 7237,
 22833,
 18182,
 9373,
 5755,
 22822,
 10715,
 18647,
 11834,
 18552,
 17216,
 23988,
 15104,
 8410,
 923,
 5722,
 21006,
 23691,
 9665,
 4836,
 22378,
 19310,
 5419,
 6416,
 17695,
 23571,
 18590,
 5220,
 4517,
 18169,
 18741,
 24229,
 18084,
 19569,
 5221,
 7977,
 23607,
 12081,
 18026,
 5996,
 11422,
 7709,
 18465,
 9854,
 329,
 10326,
 3526,
 18116,
 19163,
 13618,
 6662,
 19749,
 22281,
 20804,
 18384,
 15633,
 18777,
 4489,
 21537,
 22179,
 15146,
 24314,
 8542,
 11484,
 18656,
 10537,
 18897,
 1365,
 10727,
 15984,
 1330,
 11083,
 10762,
 8852,
 20937,
 13046,
 16093,
 9605,
 23190,
 21775,
 23851,
 15840,
 18832,
 20425,
 20851,
 17589,
 10359,
 15344,
 22753,
 22511,
 22575,
 18271,
 16101,
 4650,
 22001,
 9742,
 6842,
 5670,
 1835,
 21630,
 18303,
 20430,
 11694,
 21949,
 21371,
 7795,
 5444,
 12818,
 16328,
 15930,
 7174,
 19665,
 15039,
 15582,
 22743,
 6134,
 12343,
 13423,
 20277,
 23336,
 20888,
 6144,
 8553,
 21198,
 14851,
 7160,
 15462,
 20015,
 20274,
 16144,
 720,
 19162,
 1829,
 12649,
 12971,
 7624,
 22259,
 3089,
 11141,
 24238,
 6368,
 11275,
 21784,
 14733,
 4838,
 15958,
 8313,
 15605,
 24282,
 23518,
 10476,
 3497,
 17114,
 14740,
 13843,
 23292,
 11287,
 7005,
 21813,
 12695,
 8094,
 13402,
 8439,
 4968,
 14543,
 135,
 3963,
 18904,
 20882,
 3566,
 15198,
 4647,
 7176,
 23613,
 6727,
 7116,
 3865,
 3202,
 5816,
 24263,
 22593,
 23616,
 6942,
 14569,
 9087,
 14762,
 18148,
 11115,
 16942,
 11620,
 711,
 22601,
 14873,
 19233,
 8460,
 7545,
 17573,
 6396,
 16261,
 9262,
 22000,
 8329,
 11380,
 14702,
 5322,
 5756,
 2225]

In [9]:
bad_news = [12669,
 17432,
 803,
 8411,
 22076,
 5050,
 4846,
 7437,
 12361,
 4605,
 9525,
 20360,
 799,
 21230,
 13799,
 6265,
 8684,
 626,
 298,
 20989,
 22689,
 5993,
 2892,
 4106,
 8123,
 23233,
 7127,
 22645,
 11882,
 6572,
 20231,
 13295,
 10829,
 3716,
 20687,
 7374,
 12245,
 8696,
 4078,
 9882,
 22952,
 650,
 19713,
 2732,
 6749,
 23740,
 6741,
 18865,
 3697,
 21345,
 14376,
 20519,
 13672,
 56,
 7339,
 12180,
 23189,
 17863,
 4289,
 3910,
 7107,
 7716,
 9767,
 16106,
 20755,
 15926,
 7128,
 2281,
 927,
 23268,
 7857,
 532,
 23422,
 5777,
 5389,
 19826,
 217,
 21292,
 23857,
 12365,
 3475,
 4806,
 6101,
 7542,
 930,
 5814,
 15721,
 2662,
 105,
 12325,
 3673,
 24577,
 21425,
 4521,
 4235,
 23045,
 21472,
 15517,
 23134,
 8932,
 1617,
 4342,
 20594,
 23796,
 21000,
 12535,
 21240,
 12482,
 2631,
 17343,
 18580,
 10702,
 24507,
 10315,
 14317,
 12809,
 19583,
 22710,
 5791,
 13035,
 3681,
 10701,
 16927,
 18032,
 20704,
 1230,
 13251,
 10212,
 19271,
 1118,
 16097,
 24580,
 411,
 5836,
 11580,
 1464,
 16645,
 2586,
 10647,
 7916,
 2614,
 6641,
 17092,
 4250,
 20815,
 16970,
 12056,
 9664,
 2046,
 15690,
 10947,
 23472,
 8170,
 23848,
 1688,
 10803,
 16531,
 7056,
 21593,
 18541,
 9290,
 6628,
 13051,
 543,
 21814,
 205,
 16044,
 18586,
 10129,
 12519,
 23568,
 4884,
 22005,
 13197,
 16275,
 17337,
 336,
 7173,
 384,
 17521,
 17335,
 11627,
 10791,
 20354,
 3847,
 4358,
 13892,
 21053,
 16619,
 450,
 24034,
 7443,
 10421,
 2984,
 19041,
 5286,
 10603,
 19205,
 1598,
 5983,
 4633,
 14882,
 24363,
 7567,
 5029,
 8247,
 2206,
 20331,
 14808,
 1486,
 8321,
 4979,
 5832,
 19703,
 4262,
 3279,
 9277,
 9612,
 11415,
 17742,
 23749,
 19639,
 3875,
 22369,
 10837,
 15442,
 18393,
 16504,
 16582,
 9835,
 2319,
 6680,
 10724,
 21323,
 21559,
 5949,
 9216,
 12363,
 5611,
 23875,
 10031,
 7384,
 5005,
 12396,
 2403,
 7989,
 14101,
 14699,
 20990,
 4398,
 740,
 13062,
 5395,
 7123,
 20806,
 17281,
 15616,
 7139,
 18567,
 4566,
 11088,
 14419,
 9226,
 13456,
 11300,
 5998,
 7873,
 23355,
 21396,
 4202,
 1322,
 12757,
 7119,
 23944,
 17181,
 320,
 10401,
 2366,
 7806,
 18972,
 24186,
 11939,
 7051,
 421,
 209,
 16420,
 6108,
 448,
 21063,
 22446,
 23350,
 2093,
 16706,
 737,
 19798,
 10580,
 24165,
 66,
 7000,
 22878,
 20532,
 8115,
 11521,
 24514,
 11046,
 6704,
 10240,
 14480,
 12840,
 7914,
 10705,
 20270,
 1280,
 20293,
 21546,
 5766,
 741,
 13992,
 299,
 821,
 11142,
 19339,
 2289,
 17116,
 17339,
 10070,
 13623,
 7312,
 12433,
 7804,
 3201,
 9799,
 14761,
 9982,
 2000,
 17661,
 19147,
 3645,
 23337,
 9279,
 12248,
 122,
 22649,
 23694,
 7047,
 8491,
 1343,
 10147,
 15197,
 7224,
 21576,
 22087,
 6644,
 481,
 14782,
 7205,
 13318,
 19099,
 21652,
 22887,
 4603,
 20343,
 1467,
 11568,
 3567,
 265,
 20785,
 772,
 15046,
 13738,
 7288,
 7676,
 3410,
 6948,
 12085,
 2313,
 14704,
 19468,
 21390,
 11137,
 24613,
 11656,
 11055,
 6923,
 16103,
 3813,
 6780,
 11974,
 2547,
 8649,
 22507,
 6999,
 2697,
 17513,
 5569,
 17628,
 9518,
 674,
 22976,
 9447,
 9930,
 24344,
 3569,
 20048,
 22065,
 18942,
 17690,
 16957,
 9218,
 2031,
 3694,
 4090,
 21557,
 13704,
 2974,
 21249,
 15433,
 525,
 15682,
 19623,
 24524,
 2475,
 2699,
 12039,
 1628,
 9334,
 9620,
 1338,
 14613,
 24515,
 5439,
 14696,
 8584,
 16766,
 638,
 2949,
 8159,
 8954,
 1891,
 20314,
 6981,
 2376,
 5627,
 3392,
 16267,
 21620,
 5850,
 6383,
 8032,
 2405,
 7556,
 3270,
 10514,
 2521,
 12651,
 21724,
 12525,
 15335,
 3928,
 13401,
 10481,
 24467,
 13372,
 23154,
 4818,
 4622,
 23557,
 16081,
 24033,
 940,
 14492,
 2336,
 12558,
 18325,
 6813,
 8405,
 9574,
 24642,
 8939,
 15183,
 4898,
 2804,
 4131,
 23299,
 23178,
 615,
 20196,
 10597,
 15627,
 21763,
 8826,
 1561,
 12391,
 1703,
 12970,
 22394,
 5232,
 16301,
 2382,
 2126,
 10180,
 3200,
 7787,
 19783,
 2102,
 1585,
 3595,
 18082,
 2291,
 13803,
 15550,
 14512,
 533,
 8217,
 5657,
 21582,
 3556,
 3216,
 16419,
 11257,
 1070,
 10426,
 20637,
 19977,
 6397,
 18806,
 383,
 1609,
 20233,
 12832,
 17257,
 842,
 9284,
 19305,
 23143,
 15292,
 5857,
 3487,
 22890,
 11054,
 21760,
 12354,
 12112,
 19641,
 16633,
 6930,
 1957,
 1772,
 5114,
 8789,
 20546,
 18472,
 3191,
 22379,
 14544,
 3045,
 6063,
 20952,
 23716,
 3493,
 8223,
 15642,
 14630,
 16227,
 8230,
 14912,
 13473,
 14274,
 17535,
 125,
 10515,
 23513,
 16796,
 19450,
 2489,
 7643,
 14064,
 373,
 15806,
 16235,
 21906,
 5360,
 2301,
 14566,
 13875,
 21020,
 2383,
 5469,
 3048,
 15108,
 22483,
 5047,
 12477,
 538,
 17592,
 10225,
 10441,
 19995,
 877,
 16687,
 1773,
 20545,
 1313,
 3753,
 8194,
 23887,
 2837,
 18468,
 7459,
 23823,
 9275,
 23829,
 8471,
 24046,
 21942,
 6945,
 9096,
 16123,
 13811,
 823,
 8825,
 20332,
 6740,
 365,
 558,
 10733,
 10728,
 22103,
 18052,
 14546,
 6737,
 23646,
 18304,
 20349,
 16414,
 4642,
 23433,
 8290,
 2018,
 5240,
 307,
 18756,
 16429,
 1389,
 16971,
 16508,
 3920,
 22420,
 21923,
 10909,
 17902,
 15950,
 951,
 9827,
 1808,
 19895,
 3640,
 12872,
 1669,
 24174,
 10592,
 19758,
 11133,
 5885,
 14960,
 6305,
 12046,
 23559,
 19355,
 10978,
 562,
 18230,
 13617,
 13043,
 2074,
 15425,
 14390,
 7675,
 6961,
 7476,
 20045,
 2899,
 4433,
 10688,
 1700,
 4053,
 19445,
 8886,
 23884,
 3711,
 17228,
 12607,
 6281,
 10917,
 3318,
 9979,
 11062,
 23380,
 15577,
 7272,
 21372,
 12956,
 14647,
 7440,
 1685,
 18035,
 16708,
 527,
 9769,
 3861,
 7858,
 10582,
 1693,
 18995,
 16584,
 19257,
 4585,
 23575,
 19275,
 21353,
 7562,
 12621,
 11286,
 19788,
 7820,
 12980,
 13857,
 13079,
 8195,
 10311,
 5912,
 13034,
 6642,
 5040,
 23929,
 13147,
 21961,
 23198,
 16624,
 11166,
 3492,
 18276,
 21244,
 23729,
 22232,
 3243,
 13183,
 1267,
 8691,
 5873,
 1837,
 8889,
 378,
 13434,
 6464,
 15819,
 22007,
 4902,
 7361,
 3652,
 15964,
 18301,
 11864,
 24190,
 6596,
 255,
 6075,
 17503,
 20216,
 6817,
 16111,
 11027,
 12644,
 24438,
 9368,
 1321,
 7444,
 19372,
 17930,
 7980,
 699,
 4396,
 19728,
 11821,
 24184,
 6040,
 10591,
 567,
 10013,
 21050,
 17664,
 47,
 14898,
 8578,
 8812,
 6746]

In [11]:
# добавочные новости для train. пул новостей при активном обучении
indexes = data.index.tolist()

add_news = []

for index in indexes:
  if index not in good_news and index not in bad_news:
    add_news.append(index)

In [12]:
data_good_news = data.loc[good_news]
data_good_news['relevance'] = 1

data_bad_news = data.loc[bad_news]
data_bad_news['relevance'] = 0

In [13]:
data_for_predict = data.loc[add_news]
data_for_predict.to_csv('data_for_predict.csv', index=False, encoding='utf-8')

In [14]:
load_data = load_dataset("csv", data_files={'train': '/content/data_for_predict.csv'})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-3eb78b655b7af301/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

model = AutoModelForSequenceClassification.from_pretrained("/content/bald")

training_args = TrainingArguments(
    output_dir=".",
    num_train_epochs=2,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = BaalTransformersTrainer(
    model=model,
    args=training_args,
    tokenizer=None
)

In [16]:
def tokenize_function(examples):
    return tokenizer(examples['name_body'], max_length=200, truncation=True, padding="max_length")

In [17]:
tokenized_data = load_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/23102 [00:00<?, ? examples/s]

In [18]:
output = trainer.predict(tokenized_data['train'])

In [19]:
predict_labels = np.argmax(output.predictions, axis=1)

In [20]:
data_for_predict['relevance'] = predict_labels
data_for_predict

,id,name,ctime,body,match_id,name_body,relevance
0,7006398,Хиддинк постетит матч «Астон Виллы» и ЦСКА,2009-02-18 03:17:23,Главный тренер сборной России и «Челси» Гус Хи...,18936,Хиддинк постетит матч «Астон Виллы» и ЦСКА. Гл...,0
1,7010916,Кубок УЕФА. 1/16 финала. Первые матчи. «Зенит»...,2009-02-18 15:57:00,Кубок УЕФА 1/16 финала Первые матчи,18935,Кубок УЕФА. 1/16 финала. Первые матчи. «Зенит»...,0
2,7010916,Кубок УЕФА. 1/16 финала. Первые матчи. «Зенит»...,2009-02-18 15:57:00,Кубок УЕФА 1/16 финала Первые матчи,18936,Кубок УЕФА. 1/16 финала. Первые матчи. «Зенит»...,0
3,7012099,«Зенит» и «Штутгарт» могут сыграть красным мяч...,2009-02-18 17:33:12,Делегат УЕФА Зуне Хелльстремер сегодня утром о...,18935,«Зенит» и «Штутгарт» могут сыграть красным мяч...,0
4,7012532,В кассах «Петровского» осталось 1500 билетов н...,2009-02-18 18:03:35,"Как стало известно корреспонденту Sports.ru, н...",18935,В кассах «Петровского» осталось 1500 билетов н...,0
...,...,...,...,...,...,...,...
24643,1040626745,НХЛ. «Питтсбург» – «Сан-Хосе». Ньето заменит З...,2016-06-01 20:52:37,Нападающий «Сан-Хосе» Мэттью Ньето войдет в со...,1104866,НХЛ. «Питтсбург» – «Сан-Хосе». Ньето заменит З...,0
24644,1040636713,НХЛ. Форвард «Питтсбурга» Шери забил первый по...,2016-06-02 08:20:10,Во втором матче финала Кубка Стэнли «Питтсбург...,1104866,НХЛ. Форвард «Питтсбурга» Шери забил первый по...,1
24645,1040636981,НХЛ. Мюррей одержал 13-ю победу в дебютном пле...,2016-06-02 08:41:16,Во втором матче финала Кубка Стэнли «Питтсбург...,1104866,НХЛ. Мюррей одержал 13-ю победу в дебютном пле...,1
24646,1040637171,НХЛ. Звено Хагелин – Бонино – Кессел суммарно ...,2016-06-02 08:52:19,Во втором матче финала Кубка Стэнли «Питтсбург...,1104866,НХЛ. Звено Хагелин – Бонино – Кессел суммарно ...,1


In [21]:
resul_data = pd.concat([data_good_news, data_bad_news, data_for_predict])
resul_data

,id,name,ctime,body,match_id,name_body,relevance
1060,70098967,Денисов получил травму в матче с «Локомотивом»,2010-04-11 15:35:05,Полузащитник «Зенита» Игорь Денисов получил по...,32552,Денисов получил травму в матче с «Локомотивом»...,1
22621,1035039233,«Анжи» – ЦСКА – 1:1. Максимов спас хозяев от п...,2015-11-29 09:28:46,Сегодня в 17-м туре чемпионата России ЦСКА на ...,1035349,«Анжи» – ЦСКА – 1:1. Максимов спас хозяев от п...,1
11364,149063708,«Чикаго» – «Детройт». Кэйн забросил первую шай...,2013-05-18 21:36:16,Форвард «Чикаго» Патрик Кэйн открыл счет во вт...,780500,«Чикаго» – «Детройт». Кэйн забросил первую шай...,1
14128,157992899,Финляндия – Россия. Бобровский заменил Варламо...,2014-02-19 17:39:35,По ходу второго периода матча 1/4 финала олимп...,829733,Финляндия – Россия. Бобровский заменил Варламо...,1
19302,1027781272,Роббен одержал 100-ю победу в бундеслиге и уст...,2015-03-07 19:30:14,В матче 24-го тура чемпионата Германии «Бавари...,965242,Роббен одержал 100-ю победу в бундеслиге и уст...,1
...,...,...,...,...,...,...,...
24643,1040626745,НХЛ. «Питтсбург» – «Сан-Хосе». Ньето заменит З...,2016-06-01 20:52:37,Нападающий «Сан-Хосе» Мэттью Ньето войдет в со...,1104866,НХЛ. «Питтсбург» – «Сан-Хосе». Ньето заменит З...,0
24644,1040636713,НХЛ. Форвард «Питтсбурга» Шери забил первый по...,2016-06-02 08:20:10,Во втором матче финала Кубка Стэнли «Питтсбург...,1104866,НХЛ. Форвард «Питтсбурга» Шери забил первый по...,1
24645,1040636981,НХЛ. Мюррей одержал 13-ю победу в дебютном пле...,2016-06-02 08:41:16,Во втором матче финала Кубка Стэнли «Питтсбург...,1104866,НХЛ. Мюррей одержал 13-ю победу в дебютном пле...,1
24646,1040637171,НХЛ. Звено Хагелин – Бонино – Кессел суммарно ...,2016-06-02 08:52:19,Во втором матче финала Кубка Стэнли «Питтсбург...,1104866,НХЛ. Звено Хагелин – Бонино – Кессел суммарно ...,1


In [22]:
resul_data['relevance'].value_counts()

1    12838
0    11810
Name: relevance, dtype: int64

In [23]:
resul_data.to_csv('resul_data.csv', index=False, encoding='utf-8')

In [24]:
!cp "/content/resul_data.csv" "/content/drive/MyDrive/RuSportSum/resul_data.csv"